# AGC + calver coffea on coffea-casa

We'll base this on a few sources:
- https://github.com/iris-hep/analysis-grand-challenge/tree/main/analyses/cms-open-data-ttbar (AGC, of course)
- https://github.com/alexander-held/CompHEP-2023-AGC (contains a simplified version of AGC)
- https://github.com/nsmith-/TTGamma_LongExercise/ (credit Nick Smith for helpful examples of the new API)
- (and if time allows, weight features: https://github.com/CoffeaTeam/coffea/blob/backports-v0.7.x/binder/accumulators.ipynb / https://coffeateam.github.io/coffea/api/coffea.analysis_tools.Weights.html#coffea.analysis_tools.Weights.partial_weight)

In [1]:
from pathlib import Path

import awkward as ak
import dask
import dask_awkward as dak
import hist.dask
import coffea
import numpy as np
import uproot
import traceback
from dask.distributed import Client
import skhep_testdata

from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import PackedSelection
from coffea import dataset_tools

import warnings

import utils
utils.plotting.set_style()

warnings.filterwarnings("ignore")
NanoAODSchema.warn_missing_crossrefs = False # silences warnings about branches we will not use here


client = Client("tls://localhost:8786")

print(f"awkward: {ak.__version__}")
print(f"dask-awkward: {dak.__version__}")
print(f"dask: {dask.__version__}")
print(f"uproot: {uproot.__version__}")
print(f"hist: {hist.__version__}")
print(f"coffea: {coffea.__version__}")

/usr/local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


awkward: 2.6.3
dask-awkward: 2024.3.0
dask: 2024.7.1
uproot: 5.3.10
hist: 2.7.2
coffea: 2024.6.1


### Produce an AGC histogram with Dask (no coffea yet)

In [2]:
def calculate_trijet_mass(events):
    # pT > 30 GeV for leptons, > 25 GeV for jets
    selected_electrons = events.Electron[events.Electron.pt > 30 & (np.abs(events.Electron.eta) < 2.1)]
    selected_muons = events.Muon[events.Muon.pt > 30 & (np.abs(events.Muon.eta) < 2.1)]
    selected_jets = events.Jet[events.Jet.pt > 25 & (np.abs(events.Jet.eta) < 2.4)]

    # single lepton requirement
    event_filters = ((ak.count(selected_electrons.pt, axis=1) + ak.count(selected_muons.pt, axis=1)) == 1)
    # at least four jets
    event_filters = event_filters & (ak.count(selected_jets.pt, axis=1) >= 4)
    # at least two b-tagged jets ("tag" means score above threshold)
    B_TAG_THRESHOLD = 0.5
    event_filters = event_filters & (ak.sum(selected_jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) >= 2)

    # apply filters
    selected_jets = selected_jets[event_filters]

    trijet = ak.combinations(selected_jets, 3, fields=["j1", "j2", "j3"])  # trijet candidate
    trijet["p4"] = trijet.j1 + trijet.j2 + trijet.j3  # four-momentum of tri-jet system

    trijet["max_btag"] = np.maximum(trijet.j1.btagCSVV2, np.maximum(trijet.j2.btagCSVV2, trijet.j3.btagCSVV2))
    trijet = trijet[trijet.max_btag > B_TAG_THRESHOLD]  # at least one-btag in trijet candidates
    # pick trijet candidate with largest pT and calculate mass of system
    trijet_mass = trijet["p4"][ak.argmax(trijet.p4.pt, axis=1, keepdims=True)].mass
    return ak.flatten(trijet_mass)

def calculate_trijet_mass_rntuple(events): # TODO: implement
    # pT > 30 GeV for leptons, > 25 GeV for jets
    selected_electrons = events.Electron[events.Electron.pt > 30 & (np.abs(events.Electron.eta) < 2.1)]
    selected_muons = events.Muon[events.Muon.pt > 30 & (np.abs(events.Muon.eta) < 2.1)]
    selected_jets = events.Jet[events.Jet.pt > 25 & (np.abs(events.Jet.eta) < 2.4)]

    # single lepton requirement
    event_filters = ((ak.count(selected_electrons.pt, axis=1) + ak.count(selected_muons.pt, axis=1)) == 1)
    # at least four jets
    event_filters = event_filters & (ak.count(selected_jets.pt, axis=1) >= 4)
    # at least two b-tagged jets ("tag" means score above threshold)
    B_TAG_THRESHOLD = 0.5
    event_filters = event_filters & (ak.sum(selected_jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) >= 2)

    # apply filters
    selected_jets = selected_jets[event_filters]

    trijet = ak.combinations(selected_jets, 3, fields=["j1", "j2", "j3"])  # trijet candidate
    trijet["p4"] = trijet.j1 + trijet.j2 + trijet.j3  # four-momentum of tri-jet system

    trijet["max_btag"] = np.maximum(trijet.j1.btagCSVV2, np.maximum(trijet.j2.btagCSVV2, trijet.j3.btagCSVV2))
    trijet = trijet[trijet.max_btag > B_TAG_THRESHOLD]  # at least one-btag in trijet candidates
    # pick trijet candidate with largest pT and calculate mass of system
    trijet_mass = trijet["p4"][ak.argmax(trijet.p4.pt, axis=1, keepdims=True)].mass
    return ak.flatten(trijet_mass)

Reading in the ROOT file, we can now create a Dask task graph for the calculations and plot that we want to make using `dask-awkward` and `hist.dask`

In [3]:
# # define work to be done
# def uproot_open_materialize(fname):
#     BRANCH_LIST = [
#         "GenPart_pt", "GenPart_eta", "GenPart_phi", "CorrT1METJet_phi",
#         "GenJet_pt", "CorrT1METJet_eta", "SoftActivityJet_pt",
#         "Jet_eta", "Jet_phi", "SoftActivityJet_eta", "SoftActivityJet_phi", 
#         "CorrT1METJet_rawPt", "Jet_btagDeepFlavB", "GenJet_eta", 
#         "GenPart_mass", "GenJet_phi",
#         "Jet_puIdDisc", "CorrT1METJet_muonSubtrFactor", "Jet_btagDeepFlavCvL",
#         "Jet_btagDeepFlavQG", "Jet_mass", "Jet_pt", "GenPart_pdgId",
#         "Jet_btagDeepFlavCvB", "Jet_cRegCorr"
#         ]

#     filter_name = lambda x: x in BRANCH_LIST

#     size_uncompressed = 0
#     t0 = time.perf_counter()
#     try:
#         with uproot.open(fname, filter_name=filter_name) as f:
#             num_entries = f["Events"].num_entries
#             for b in BRANCH_LIST:
#                 f["Events"][b].array()
#                 size_uncompressed += f["Events"][b].uncompressed_bytes

#             size_read = f.file.source.num_requested_bytes
#         exception = None

#     except:
#         num_entries = 0
#         size_read = 0
#         size_uncompressed = 0
#         exception = traceback.format_exc()

#     t1 = time.perf_counter()
#     time_finished = datetime.datetime.now()
#     return {"fname": fname, "read": size_read, "uncompressed": size_uncompressed, "num_entries": num_entries,
#             "runtime": t1-t0, "time_finished": time_finished, "exception": exception}

In [4]:
ttbar_file = "https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/"\
    "TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19981_PU25nsData2015v1_76X_"\
    "mcRun2_asymptotic_v12_ext4-v1_80000_0007.root" # 533M size

# Choose RNTuple file instead:
# rntuple_dirs = []
# rntuple_files = []
# TTbar original: root://eospublic.cern.ch//eos/root-eos/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19981_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext4-v1_80000_0007.root # 533M size
# root://eospublic.cern.ch//eos/root-eos/AGC/rntuple/nanoAOD/TT_TuneCUETP8M1_13TeV-amcatnlo-pythia8/cmsopendata2015_ttbar_19978_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext1-v1_60000_0004.root
# root://eospublic.cern.ch//eos/root-eos/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-amcatnlo-pythia8/cmsopendata2015_ttbar_19978_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext1-v1_60000_0004.root



# xrd_fpath = "root://eospublic.cern.ch//eos/root-eos/AGC"
# # Smallest files are chosen, because big ones are crashing in computation step.
# data_path_ends = [
# "/nanoAOD/TT_TuneCUETP8M1_13TeV-amcatnlo-pythia8/cmsopendata2015_ttbar_19978_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext1-v1_60000_0004.root", # it is 451 MB size,
# "/rntuple/nanoAOD/TT_TuneCUETP8M1_13TeV-amcatnlo-pythia8/cmsopendata2015_ttbar_19978_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext1-v1_60000_0004.root"
# ]
# for path_end in data_path_ends:
#     rntuple_files.append(xrd_fpath + path_end)
    
# # List all files in each directory
# for dir_path in rntuple_dirs:
#     print(f"Getting files from: {dir_path}")
#     list_files_in_directory(dir_path)

# Choose files for analysis:
# all_files = rntuple_files
# all_files.append("/home/cms-jovyan/.local/skhepdata/Run2012BC_DoubleMuParked_Muons_rntuple_1000evts.root/scikit-hep-scikit-hep-testdata-9ce1f2b/src/skhep_testdata/data/Run2012BC_DoubleMuParked_Muons_rntuple_1000evts.root")
# print(skhep_testdata.data_path("Run2012BC_DoubleMuParked_Muons_rntuple_1000evts.root"))


# File dir:  root://eospublic.cern.ch//eos/root-eos/AGC/eos/root/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-amcatnlo-pythia8/
# File dir2:  root://eospublic.cern.ch//eos/root-eos/AGC/eos/root/AGC/rntuple/nanoAOD/TT_TuneCUETP8M1_13TeV-amcatnlo-pythia8/

In [5]:

# if chosen_file == rntuple_files[0]:
# try:
#     with uproot.open(chosen_file) as f:
#         num_entries = f["Events"].num_entries
#         events = f["Events"]
#         print("Events len: ", len(events))
#     exception = None
# except:
#     events = None
#     num_entries = 0
#     size_read = 0
#     size_uncompressed = 0
#     exception = traceback.format_exc()
#     print(exception)
# else: # TTBar file case:
# for file in all_files:
#     print("File to analyze: ", file)
    # One way of opening the file (coffea approach):
    # print(events.fields)

# # TTree file analysis:
# events1 = NanoEventsFactory.from_root({all_files[0]: "Events"}, schemaclass=NanoAODSchema).events()
# selected_electrons = events1.Electron[events1.Electron.pt > 30 & (np.abs(events1.Electron.eta) < 2.1)]
# selected_muons = events1.Muon[events1.Muon.pt > 30 & (np.abs(events1.Muon.eta) < 2.1)]
# selected_jets = events1.Jet[events1.Jet.pt > 25 & (np.abs(events1.Jet.eta) < 2.4)]

# Files are downloaded locally:
all_files = []
all_files.append("/home/cms-jovyan/my_root_files/ttree/cmsopendata2015_ttbar_19978_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext1-v1_60000_0004.root")
all_files.append("/home/cms-jovyan/my_root_files/ttree/cmsopendata2015_ttbar_19981_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext4-v1_80000_0007.root")
all_files.append("/home/cms-jovyan/my_root_files/rntuple/cmsopendata2015_ttbar_19978_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext1-v1_60000_0004.root") # RNTuple

with uproot.open(all_files[0]) as f: # Remote TTree
    events1 = f["Events"]
    print("File was loaded, event count: ", len(events1.keys()))

with uproot.open(all_files[1]) as f:
    events2 = f["Events"]
    print("File was loaded, event count: ", len(events2.keys()))
    
with uproot.open(all_files[2]) as f:
    events3= f["Events"]
    print("File was loaded, event count: ", len(events3.keys()))
    




    



File was loaded, event count:  947
File was loaded, event count:  947
File was loaded, event count:  947


In [10]:
## Test times for each function:

# import timeit

# # Define the operations as functions
# def array_1():
#     print(events1.arrays()["Electron_pt"])

# def array_1_direct():
#     print(events1["Electron_pt"].array())

# def array_2():
#     print(events2.arrays()["Electron_pt"])

# def array_2_direct():
#     print(events2["Electron_pt"].array())

# def array_3():
#     print(events3.arrays()["Electron_pt"])

# def array_3_direct():
#     print(events3["Electron_pt"].array())

# # Time the operations
# time_1 = timeit.timeit(array_1, number=1)
# print("Time for events1.arrays()['Electron_pt']: ", time_1)
# time_1_direct = timeit.timeit(array_1_direct, number=1)
# print("Time for events1['Electron_pt'].array(): ", time_1_direct)

# print("*****")

# time_2 = timeit.timeit(array_2, number=1)
# print("Time for events2.arrays()['Electron_pt']: ", time_2)
# time_2_direct = timeit.timeit(array_2_direct, number=1)
# print("Time for events2['Electron_pt'].array(): ", time_2_direct)

# print("*****")

# time_3 = timeit.timeit(array_3, number=1)
# print("Time for events3.arrays()['Electron_pt']: ", time_3)
# time_3_direct = timeit.timeit(array_3_direct, number=1)
# print("Time for events3['Electron_pt'].array(): ", time_3_direct)








[[], [9.65], [61.6, 25.7, 23.9, 8.55], [79.9], ..., [...], [7.32], [15.8], []]
Time for events1.arrays()['Electron_pt']:  43.30151801696047
[[], [9.65], [61.6, 25.7, 23.9, 8.55], [79.9], ..., [...], [7.32], [15.8], []]
Time for events1['Electron_pt'].array():  0.014032426988705993
*****
[[], [], [], [], [17.1, 8.75, 5.49], [], ..., [...], [30.6], [], [34.3], [23.8]]
Time for events2.arrays()['Electron_pt']:  43.88196036010049
[[], [], [], [], [17.1, 8.75, 5.49], [], ..., [...], [30.6], [], [34.3], [23.8]]
Time for events2['Electron_pt'].array():  0.01606880221515894
*****
[[], [9.65], [61.6, 25.7, 23.9, 8.55], ..., [11.6], [86.6], [33.7, 30.9]]
Time for events3.arrays()['Electron_pt']:  4.742254454875365


TypeError: 'Model_ROOT_3a3a_Experimental_3a3a_RNTuple' object is not subscriptable

In [11]:
print(events3.arrays()['Electron_pt'])


[[], [9.65], [61.6, 25.7, 23.9, 8.55], ..., [11.6], [86.6], [33.7, 30.9]]


In [ ]:
# create the task graph to build a histogram
print("Calculating trijet mass...")
reconstructed_top_mass = calculate_trijet_mass(events)
print("hist_reco_mtop...")
hist_reco_mtop = hist.dask.Hist.new.Reg(16, 0, 375, label="$m_{bjj}$").Double().fill(reconstructed_top_mass)

and then once we're ready we can execute the task graph with `.compute()` to get our visualization

In [ ]:
# perform computation and visualize
artists = hist_reco_mtop.compute().plot()

In [ ]:
# and annotate the visualization
fig_dir = Path.cwd() / "figures"
fig_dir.mkdir(parents=True, exist_ok=True)

ax = artists[0].stairs.axes
fig = ax.get_figure()
ax.vlines(175, 0, 10000, colors=["grey"], linestyle="dotted")
ax.text(180, 150, "$m_{t} = 175$ GeV")
ax.set_xlim([0, 375])
ax.set_ylim([0, 8000])

fig.savefig(fig_dir / "trijet_mass.png", dpi=300)
fig

This all matches the (non-Dask) versions of the plots from last summer — see the notebook linked above. Not surprising, but reassuring!

### Time for coffea

We'll first write the functions to compute the observable and do the histogramming using `awkward-dask` and `hist.dask` again

In [ ]:
B_TAG_THRESHOLD = 0.5

# perform object selection
def object_selection(events):
    elecs = events.Electron
    muons = events.Muon
    jets = events.Jet

    electron_reqs = (elecs.pt > 30) & (np.abs(elecs.eta) < 2.1) & (elecs.cutBased == 4) & (elecs.sip3d < 4)
    muon_reqs = ((muons.pt > 30) & (np.abs(muons.eta) < 2.1) & (muons.tightId) & (muons.sip3d < 4) &
                 (muons.pfRelIso04_all < 0.15))
    jet_reqs = (jets.pt > 30) & (np.abs(jets.eta) < 2.4) & (jets.isTightLeptonVeto)

    # Only keep objects that pass our requirements
    elecs = elecs[electron_reqs]
    muons = muons[muon_reqs]
    jets = jets[jet_reqs]

    return elecs, muons, jets


# event selection for 4j1b and 4j2b
def region_selection(elecs, muons, jets):
    ######### Store boolean masks with PackedSelection ##########
    selections = PackedSelection(dtype='uint64')
    # Basic selection criteria
    selections.add("exactly_1l", (ak.num(elecs) + ak.num(muons)) == 1)
    selections.add("atleast_4j", ak.num(jets) >= 4)
    selections.add("exactly_1b", ak.sum(jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) == 1)
    selections.add("atleast_2b", ak.sum(jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) >= 2)
    # Complex selection criteria
    selections.add("4j1b", selections.all("exactly_1l", "atleast_4j", "exactly_1b"))
    selections.add("4j2b", selections.all("exactly_1l", "atleast_4j", "atleast_2b"))

    return selections.all("4j1b"), selections.all("4j2b")


# observable calculation for 4j2b
def calculate_m_reco_top(jets):
    # reconstruct hadronic top as bjj system with largest pT
    trijet = ak.combinations(jets, 3, fields=["j1", "j2", "j3"])  # trijet candidates
    trijet["p4"] = trijet.j1 + trijet.j2 + trijet.j3  # four-momentum of tri-jet system
    trijet["max_btag"] = np.maximum(trijet.j1.btagCSVV2,
                                    np.maximum(trijet.j2.btagCSVV2, trijet.j3.btagCSVV2))
    trijet = trijet[trijet.max_btag > B_TAG_THRESHOLD]  # at least one-btag in candidates
    # pick trijet candidate with largest pT and calculate mass of system
    trijet_mass = trijet["p4"][ak.argmax(trijet.p4.pt, axis=1, keepdims=True)].mass
    observable = ak.flatten(trijet_mass)

    return observable


# create histograms with observables
def create_histograms(events):
    hist_4j1b = (
        hist.dask.Hist.new.Reg(25, 50, 550, name="HT", label=r"$H_T$ [GeV]")
        .StrCat([], name="process", label="Process", growth=True)
        .StrCat([], name="variation", label="Systematic variation", growth=True)
        .Weight()
    )

    hist_4j2b = (
        hist.dask.Hist.new.Reg(25, 50, 550, name="m_reco_top", label=r"$m_{bjj}$ [GeV]")
        .StrCat([], name="process", label="Process", growth=True)
        .StrCat([], name="variation", label="Systematic variation", growth=True)
        .Weight()
    )

    process = events.metadata["process"]  # "ttbar" etc.
    variation = events.metadata["variation"]  # "nominal" etc.
    process_label = events.metadata["process_label"]  # nicer LaTeX labels

    # normalization for MC
    x_sec = events.metadata["xsec"]
    nevts_total = events.metadata["nevts"]
    lumi = 3378 # /pb
    if process != "data":
        xsec_weight = x_sec * lumi / nevts_total
    else:
        xsec_weight = 1

    elecs, muons, jets = object_selection(events)

    # region selection
    selection_4j1b, selection_4j2b = region_selection(elecs, muons, jets)

    # 4j1b: HT
    observable_4j1b = ak.sum(jets[selection_4j1b].pt, axis=-1)
    hist_4j1b.fill(observable_4j1b, weight=xsec_weight, process=process_label, variation=variation)

    # 4j2b: m_reco_top
    observable_4j2b = calculate_m_reco_top(jets[selection_4j2b])
    hist_4j2b.fill(observable_4j2b, weight=xsec_weight, process=process_label, variation=variation)

    return {"4j1b": hist_4j1b, "4j2b": hist_4j2b}

and prepare the fileset we need

In [ ]:
# fileset preparation
N_FILES_MAX_PER_SAMPLE = 1
# compared to coffea 0.7: list of file paths becomes list of dicts (path: trename)
fileset = utils.file_input.construct_fileset(N_FILES_MAX_PER_SAMPLE)

# fileset = {"ttbar__nominal": fileset["ttbar__nominal"]}  # to only process nominal ttbar
# fileset

Now we can start using `coffea` with its Dask capabilities. One of the things we need to do is to build the full task graph, which requires looping over all the sample variations (`samples`)

In [ ]:
%%time
# pre-process
samples, _ = dataset_tools.preprocess(fileset, step_size=250_000)

# workaround for https://github.com/CoffeaTeam/coffea/issues/1050 (metadata gets dropped, already fixed)
for k, v in samples.items():
    v["metadata"] = fileset[k]["metadata"]

In [ ]:
%%time
# create the task graph
tasks = dataset_tools.apply_to_fileset(create_histograms, samples, uproot_options={"allow_read_errors_with_report": True})

and then we can finally execute the full task graph with Dask

In [ ]:
%%time
# execute
((out, report),) = dask.compute(tasks)  # feels strange that this is a tuple-of-tuple

print(f"total time spent in uproot reading data (or some related metric?): {ak.sum([v['duration'] for v in report.values()]):.2f} s")

To visualize the results, we need to first stack the serperate histograms that were computed individually

In [ ]:
# stack all the histograms together (we processed each sample separately)
full_histogram_4j1b = sum([v["4j1b"] for v in out.values()])
full_histogram_4j2b = sum([v["4j2b"] for v in out.values()])

In [ ]:
artists = full_histogram_4j1b[120j::hist.rebin(2), :, "nominal"].stack("process")[::-1].plot(
    stack=True, histtype="fill", linewidth=1,edgecolor="grey"
)

ax = artists[0].stairs.axes
fig = ax.get_figure()

ax.legend(frameon=False)
ax.set_title(">= 4 jets, 1 b-tag");

fig.savefig(fig_dir / "coffea_4j_1b.png", dpi=300)

In [ ]:
artists = full_histogram_4j2b[:, :, "nominal"].stack("process")[::-1].plot(
    stack=True, histtype="fill", linewidth=1,edgecolor="grey"
)

ax = artists[0].stairs.axes
fig = ax.get_figure()

ax.legend(frameon=False)
ax.set_title(">= 4 jets, >= 2 b-tags");

fig.savefig(fig_dir / "coffea_4j_2b.png", dpi=300)

This is a plot you can compare to the one in the full AGC notebook — you'll notice they look the same. Success!

If we now investigate the task graph for the nominal $t\bar{t}$ sample in the optimzied view, which hides from us some of the complexity of the graph we created.

In [ ]:
tasks[0]["ttbar__nominal"]["4j2b"].visualize(optimize_graph=True)

In [ ]:
# "100 layers is a large task graph" on IRIS-HEP Slack, 100 layers happen quickly!
for region in ["4j1b", "4j2b"]:
    for process, task in tasks[0].items():
        print(f"{process:>30} {region} {len(task[region].dask.layers)}")

In [ ]:
# columns getting read for a given task
dak.necessary_columns(tasks[0]["ttbar__nominal"]["4j2b"])